＃車両カタログデータ

https://kakaku.com/kuruma/used/maker/%83g%83%88%83%5E/%83%89%83%93%83h%83N%83%8B%81%5B%83U%81%5B/    ランドクルーザー
https://kakaku.com/kuruma/used/maker/%83g%83%88%83%5E/%83%89%83%93%83h%83N%83%8B%81%5B%83U%81%5B%83v%83%89%83h/　　ランドクルーザープラド
https://kakaku.com/kuruma/used/maker/%83g%83%88%83%5E/%83A%83%8B%83t%83%40%81%5B%83h/　アルファード
https://kakaku.com/kuruma/used/maker/%83%8C%83N%83T%83X/RX/ 
https://kakaku.com/kuruma/used/maker/%83%81%83%8B%83Z%83f%83X%81E%83x%83%93%83c/C%83N%83%89%83X%83X%83e%81%5B%83V%83%87%83%93%83%8F%83S%83%93/   c_class_station
https://kakaku.com/kuruma/used/maker/%83X%83Y%83L/%83W%83%80%83j%81%5B/ シエラ
https://kakaku.com/kuruma/used/maker/%83%7D%83c%83%5F/%83%8D%81%5B%83h%83X%83%5E%81%5B/ ロードスター
https://kakaku.com/kuruma/used/maker/%83g%83%88%83%5E/%83N%83%89%83E%83%93%83A%83X%83%8A%81%5B%83g/ クラウンアスリーt

In [ ]:
import pyperclip

def get_clipboard_content_as_list():
    content = pyperclip.paste()  # クリップボードの内容を取得
    return content.split('\n')   # 改行で分割してリストを返す

def copy_list_to_clipboard(lst):
    content = str(lst)           # リストを文字列に変換
    pyperclip.copy(content)      # クリップボードにコピー

# クリップボードからリストを取得
clipboard_list = get_clipboard_content_as_list()

# リストをクリップボードにコピー
copy_list_to_clipboard(clipboard_list)
clipboard_list


In [ ]:
import re
import pandas as pd
import requests
from time import sleep
import time
from bs4 import BeautifulSoup
from retry import retry
import mojimoji
from datetime import datetime
import matplotlib.pyplot as plt
import random
import numpy as np

@retry(tries=3, delay=5, backoff=2)
def get_html(url):
    r = requests.get(url)
    return BeautifulSoup(r.content, "html.parser")

urls = clipboard_list

for url in urls:
    try:
        print(url)
        soup = get_html(url)
        sleep_time = random.uniform(5, 35)
        time.sleep(sleep_time)


        grade_boxes = soup.select('.gradeBox')
        model_names = [model.text.strip() for model in soup.select('.modelName .modelNameInner a')]

        dfs = []

        for idx, box in enumerate(grade_boxes):
            headers = [th.text.strip() for th in box.select('th')]
            rows = box.select('tr')[1:]
            data = []
            web_addresses = []
            
            for row in rows:
                cells = [td.text.strip() for td in row.select('td')]
                
                # 各行からwebアドレスを取得してcellsに追加
                grade_name_link = row.select_one('.gradeName a')
                web_address = grade_name_link['href'] if grade_name_link else 'N/A'
                cells.append("https://kakaku.com" + str(web_address))
                
                data.append(cells)
                
            # 列名のリストに'Web Address'を追加
            headers.append('Web Address')
            df = pd.DataFrame(data, columns=headers)
            df['model'] = model_names[idx]
            dfs.append(df)

        final_df = pd.concat(dfs, ignore_index=True)
        final_df['掲載台数'] = final_df['グレード名\xa0(掲載台数)'].str.extract(r'\((\d+)\)').astype(int)
        final_df['グレード名'] = final_df['グレード名\xa0(掲載台数)'].str.replace(r'\(\d+\)', '', regex=True).str.strip()
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        continue

    def split_price(price):
        clean_price = price.replace("万円", "").replace(",", "")
        clean_price = clean_price.replace("～", "~").replace("〜", "~")
        parts = clean_price.split("~")
        try:
            return (float(parts[0]), float(parts[1])) if len(parts) == 2 else (float(parts[0]), float(parts[0]))
        except ValueError as e:
            print(f"Error with price: {price}\nCleaned price parts: {parts}\nError message: {e}")
            return None, None
    
         
    # 1. 中古車の最小価格と最大価格を分割
    final_df['支払総額(min)'], final_df['支払総額(max)'] = zip(*final_df['支払総額'].apply(split_price))

    # 2. None値をNaNに変換
    final_df.replace({None: np.nan}, inplace=True)

    # 3. 列をfloat型に変換
    final_df['支払総額(min)'] = final_df['支払総額(min)'].astype(float)
    final_df['支払総額(max)'] = final_df['支払総額(max)'].astype(float)

    # 4. 平均を計算
    final_df['支払総額(ave)'] = ((final_df['支払総額(min)'] + final_df['支払総額(max)']) / 2).round(2)

    # 5. 平均価格がNaNの場合は空白に置き換え
    final_df['支払総額(ave)'].fillna("", inplace=True)

    # 6. 最小価格または最大価格がNaNの行を削除
    final_df.dropna(subset=['支払総額(min)', '支払総額(max)'], inplace=True)
    
    
    final_df.drop(columns=['グレード名\xa0(掲載台数)', '支払総額','定員', '駆動', '変速', '燃費'], inplace=True)
    final_df['新車価格'] = final_df['新車価格'].str.replace('万円', '').astype(float)

    final_df['支払総額(ave)'] = pd.to_numeric(final_df['支払総額(ave)'], errors='coerce')
    final_df['支払総額の割合(%)'] = ((final_df['支払総額(ave)'] / final_df['新車価格']) * 100).round(2)

    
    final_df['モデル名'] = final_df['model'].str.extract(r'^(.*?)\s\d+年モデル')
    final_df['年'] = final_df['model'].str.extract(r'(\d+年モデル)')
    # None値をNaNに変換
    # final_df.replace({None: np.nan}, inplace=True)
    if not final_df.empty:
        today_date = datetime.today().strftime('%Y%m%d')
        model_name = final_df['モデル名'].iloc[0].replace(' ', '_').replace('/', '_')
        filename = f"{today_date}_{model_name}.csv"
        final_df.to_csv(filename, index=False)
    else:
        print("DataFrame is empty!")
       
    #中古車価格の平均値は単純計算で出しているが、最頻値、中央値の算出が必要。これはそれぞれの価格を取得して分布計算させる。
    #購入した年月日と発売年月日を
print("Done!!")    

URLsのテキスト保存

In [ ]:
# with open("porsche.txt", "w") as f:
#     for url in clipboard_list:
#         f.write(url + "\n")

In [ ]:
import re
import pandas as pd
import requests
from time import sleep
from bs4 import BeautifulSoup
from retry import retry
import mojimoji

BASE_URL = "https://kakaku.com"

@retry(tries=3, delay=5, backoff=2)
def get_html(url):
    r = requests.get(url)
    return BeautifulSoup(r.content, "html.parser")

def car_model_detection(url):
    try:
        p = r"Maker=(\d{1,2})"
        p1 = r"Model=(\d{4,5})"
        maker = re.findall(p, url)
        model = re.findall(p1, url)
        return [maker[-1], model[-1]]
    except:
        return [0, 0]

def engine_category(pstr):
    p = r"(\d{3}|\d{1}.\d{1})"
    matches = re.findall(p, pstr)
    return matches[-1] if matches else 0

def parse_car_detail_info(car_url, maker):
    data = {}
    soup2 = get_html(car_url)

    data["名称"] = mojimoji.zen_to_han(soup2.find("h3").getText().strip())
    data["Class_Category"] = engine_category(data["名称"])
    data["Url"] = car_url
    data["Maker"] = maker[0]
    data["Model"] = maker[1]

    # data["価格"] = soup2.find("span", {"class": "priceTxt"}).find("span").getText().strip()
    data["総額"] = soup2.find("span", {"class": "-total"}).getText().strip()
    
    # total = re.findall(r"[0-9.]+", soup2.find("span", {"class": "total"}).getText().strip())
    data["価格"] = soup2.find("span", {"class": "-base"}).getText().strip()

    for tr in soup2.find("table", {"class": "specList"}).findAll("tr"):
        data[tr.find("th").getText().strip()] = tr.find("td").getText().strip()
    for li in soup2.find("div", {"class": "optionArea"}).findAll("li", {"class": "yes"}):
        data[li.getText().strip()] = 1
    for li in soup2.find("div", {"class": "optionArea"}).findAll("del"):
        data[li.getText().strip()] = 0
    for dt, dd in zip(soup2.find_all("dt"), soup2.find_all("dd")):
        data[dt.getText().strip()] = dd.getText().strip()

    return data

def check_next_page(soup):
    return 1 if soup.find("li", {"class": "next"}) else 0

def extract_color_from_item(item):
    color_item = item.find("li", class_=re.compile("color color\d+"))
    if color_item:
        color_class = [cls for cls in color_item["class"] if cls.startswith("color") and len(cls) > 5]
        if color_class:
            return color_class[0][5:]
    return None

def main(final_df):
    all_data = []
    originals = final_df['Web Address'].tolist()

    # for url in originals[:2]:
    for url in originals:
            # 5秒から10秒の間でランダムな時間を生成
        sleep_time = random.uniform(15, 35)
        time.sleep(sleep_time)
        print(url)
        for page in range(1, 37):
            sleep_time = random.uniform(3, 10)
            current_url = url + "Page={}/".format(page)
            soup = get_html(current_url)

            idx = final_df[final_df['Web Address'] == url].index[0]
            新車価格 = final_df.at[idx, "新車価格"]
            グレード名 = final_df.at[idx, "グレード名"]
            年 = final_df.at[idx, "年"]

            for item in soup.findAll("div", {"class": "ucItemBox"}):
                car_url = BASE_URL + item.find("a").get("href")
                detail_info = parse_car_detail_info(car_url, car_model_detection(url))

                color_info = extract_color_from_item(item)
                detail_info['color'] = color_info
                detail_info['新車価格'] = 新車価格
                detail_info['グレード名'] = グレード名
                detail_info['年'] = 年
                all_data.append(detail_info)

            if not check_next_page(soup):
                break

    return pd.DataFrame(all_data)

if __name__ == "__main__":
    dfs = main(final_df)
    print("Done!!")

#ここは2回走らせるとだめ！ 修復履歴が全て”０”になってしまう。

In [ ]:
for i in dfs.columns:
    print(i)

In [ ]:
dfs.info()

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

def remove_long_columns(df):
    remove_columns = [i for i in df.columns if len(i) >= 100]
    return df.drop(remove_columns, axis=1)

def extract_year(value):
    if isinstance(value, str) and re.search(r"[0-9]{4}", value):
        return int(re.findall(r"[0-9]{4}", value)[0])
    return np.nan

def convert_price(value):
    if value == "応談":
        return np.nan
    return float(value)

def convert_distance(value):
    value = str(value)
    if re.search(r"[0-9.]+", value):
        num = float(re.findall(r"[0-9.]+", value)[0])
        return num * 10000 if "万" in value else num
    return 0

def calculate_remaining_months(row):
    current_date = datetime.now()
    if pd.isna(row):
        return 0
    elif row in ['車検整備付', '車検整備なし']:
        return 24 if row == '車検整備付' else 0
    try:
        year, month = map(int, row.split('/'))
        target_date = datetime(year, month, 1)
        diff = (target_date.year - current_date.year) * 12 + target_date.month - current_date.month
        return diff
    except:
        return None
    
def extract_category(x):
    parts = x.split()
    return parts[1] if len(parts) > 1 else None

def preprocess_dataframe(df):
    df = remove_long_columns(df)
    
    df["年式"] = df["年式／初度登録"].apply(extract_year)
    df["価格"] = df["価格"].apply(convert_price)
    df["走行距離"] = df["走行距離"].apply(convert_distance)
    df["color"] = df["color"].astype(float)  # 追加
    df['RemainingMonths'] = df['車検'].apply(calculate_remaining_months).astype(float)
    
    df["未使用車"] = df["未使用車"].apply(lambda x: 1 if x == "○" else 0)
    df["禁煙車"] = df["禁煙車"].apply(lambda x: 1 if x == "○" else 0)
    df["ワンオーナー"] = df["ワンオーナー"].apply(lambda x: 1 if x == "○" else 0)
    df["修復歴"] = df["修復歴"].apply(lambda x: 1 if x == "修復歴あり" else 0)
    
    df = df[df["年式"] != 1991]
    df.dropna(subset=["価格", "年式"], inplace=True)
 
    df["Category"] = df.iloc[:, 0].apply(extract_category)
    df["Mission"] = df["ミッション"].apply(lambda x: 1 if x == "MT" else 0)
    
    return df


df = preprocess_dataframe(dfs)


In [ ]:
# キーワードリスト
keywords = [
    "名称",'Url',
    "color",
    '新車価格',"価格",'総額'
    "色",
    'グレード名', '年',"年式",
    "走行距離",
    "地域",
    "Mission",
    "RemainingMonths",
    "修復歴", 'ワンオーナー','エアロパーツ'
]

# keywordsに含まれる列のみを残す
cols_to_keep = [col for col in df.columns if col in keywords]
df_rev = df[cols_to_keep]  
df_rev.columns


ファイルを中間保存する。

In [ ]:
filename = f"{today_date}_{model_name}_detaildata.csv"
df_rev.to_csv(filename, index=False, encoding="utf-8-sig")


中間保存したファイルを再度読み込む場合、大規模データのscraping後はこれを使う。

In [ ]:
# import pandas as pd
# import os
# path='./20230831_ランドクルーザープラド_detaildata.csv'
# df_rev=pd.read_csv(path)
# # ファイル名（拡張子を除く）を取得
# filename = os.path.splitext(os.path.basename(path))[0]
# print(filename)

中間処理したdataframeをCSVに変換している。

https://levelup.gitconnected.com/python-libraries-for-lazy-data-scientists-c1287eb794ec

In [ ]:
# 各列の欠損値の数を取得
missing_counts = df_rev.isna().sum()
print(missing_counts)

# もし欠損値のカウントが0より大きい場合、欠損値を含む行を削除
if missing_counts.sum() > 0:
    df_rev = df_rev.dropna()
    print("欠損値を含む行を削除しました。")
else:
    print("欠損値は存在しません。")

# 削除後のデータの形状を表示
print(df_rev.shape)


SweetVIzによる可視化


In [ ]:
# 1. 必要なライブラリをインストールする
# !pip install sweetviz

# 2. ライブラリをインポートする
import pandas as pd
import sweetviz as sv
from sklearn.model_selection import train_test_split

#日本語対応
sv.config_parser.read_string("[General]\nuse_cjk_font=1")

# データセットの読み込み
# data = pd.read_csv('your_dataset.csv')

# 3. データを訓練データとテストデータに分割する
train_data, test_data = train_test_split(df_rev, test_size=0.2)

# 4. Sweetvizのanalyze関数とcompare関数を使用して、訓練データとテストデータを分析・比較する
report = sv.compare([train_data, "Training Data"], [test_data, "Test Data"])

# 5. レポートを表示する
report.show_html(f"{filename}.html")

じゅうかいき分析

In [ ]:
df_rev.columns

In [ ]:
# x = df_rev.drop(['Url',"名称","地域","Category"], axis=1)
x = df_rev.drop(['Url',"名称","地域"], axis=1)
dummies = pd.get_dummies(x, drop_first=True)
# dummies
print(np.isnan(dummies).sum(axis=0))

In [ ]:
'''
欠損値を補完する:  ここでdfの名前変更しているので、実施要。
一般的なアプローチとして、平均値や中央値、最頻値などを用いて欠損値を補完する方法があります。また、より高度な方法としては、他の特徴量を使用して欠損値を予測するモデルを訓練する方法もあります。
ここでは、SimpleImputer を使用して欠損値を補完する方法を示します。　　---これをするとnumpyになってしまう。
'''
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
dummies_imputed = imputer.fit_transform(dummies)

🎥 「特徴量の重要度を視覚化！ - ランダムフォレストを使ってみた！」

🎉 こんにちは、皆さん！今日はあるグラフについてお話ししたいと思います。このグラフ、何か特別なものかというと、実は、車の価格を予測するのに役立つ情報、つまり、どの情報が一番大切なのかを見るためのものなんです。

✨ ポイント1：バーの長さが大事！

このグラフの中で、一番気をつけてほしいのは各バーの長さです。長いバーほど、その情報が価格の予測に重要ということを示しています。簡単に言うと、長いバーを持つ項目は、車の価格を知るためのキーポイントなんです！
🚗 ポイント2：全部の情報が大事ではない！

グラフを見て分かる通り、すべての情報が同じくらい大切なわけではありません。ある情報は非常に価値がある一方で、他の情報はそれほど影響を持たないことがあります。これは、例えば、ある車の機能は価格に大きく影響するけど、別の小さな機能はそんなに影響しない、ということを示しています。
🔍 ポイント3：明確なランキングができる！

このグラフを使えば、どの情報が最も価値があるのか、次にどれが大切なのか、というように、情報の重要度をはっきりとランキングすることができます。これは、車を買う時や売る時に、どんなポイントに注目すればいいのかを知る手助けになりますね！
以上、ランダムフォレストを使って見えてきた車の価格予測に役立つ情報の重要度についてのポイントをお伝えしました！興味が湧いた方は、是非、この技術を使ってみてくださいね！🌟

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encode_dataframe(dataframe):
    result_df = dataframe.copy()
    le_dict = {}  # 各列のLabelEncoderを保存するための辞書

    # データフレーム内のオブジェクト型の列を検索
    for column in dataframe.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        result_df[column] = le.fit_transform(dataframe[column])
        le_dict[column] = le  # 逆変換のためにLabelEncoderを保存
        
    return result_df, le_dict

encoded_df, le_dict = label_encode_dataframe(dummies)
print(encoded_df)

In [ ]:
'''
ペアプロットの作成
'''

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# 日本語フォントの設定

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False


# 必要な列だけを取得
subset_df = df_rev.copy()

# ペアプロットを描画
g = sns.pairplot(data=subset_df, hue='グレード名')

# 各プロットに相関係数を追加
for i, j in zip(*np.triu_indices_from(g.axes, 1)):
    # 数値型の列のみを考慮
    if (np.issubdtype(subset_df.iloc[:, i].dtype, np.number) and
            np.issubdtype(subset_df.iloc[:, j].dtype, np.number)):
        g.axes[i, j].annotate(
            f"corr={subset_df.iloc[:, i].corr(subset_df.iloc[:, j]):.2f}",
            (0.5, 0.9), xycoords='axes fraction',
            ha='center', va='center', fontsize=10
        )

plt.show()


In [ ]:
# display(encoded_df)

In [ ]:
'''
KFold または StratifiedKFold を使用してデータを複数のトレイン/テストセットに分割します。
各分割でモデルをトレインし、特徴量の重要度を取得します。
すべての分割で取得した特徴量の重要度の平均を計算します。
平均の特徴量の重要度をプロットします。
以下は、クロスバリデーションを追加したバージョンです。
'''
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import numpy as np

# データを特徴量とターゲットに分割
X = encoded_df.drop('価格', axis=1)
Y = encoded_df['価格']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# クロスバリデーションの設定
'''
k-Fold Cross Validationは手元のデータをk個のグループに分割して,k個のうちひとつのグループをテストデータとして
残りのデータを学習データとします．それを全てのグループがテストデータになるようk回繰り返します．
'''
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 各分割での特徴量の重要度を格納するリスト
feature_importances_list = []

for train_index, test_index in kf.split(X_imputed):
    X_train, X_test = X_imputed[train_index], X_imputed[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    # ランダムフォレストモデルの作成
    rf = RandomForestRegressor(n_estimators=100, random_state=42)

    # モデルをデータに適合
    rf.fit(X_train, Y_train)

    # 特徴量の重要度を取得
    feature_importances_list.append(rf.feature_importances_)

# すべての分割での特徴量の重要度の平均を取得
average_feature_importances = np.mean(feature_importances_list, axis=0)

# 重要度の降順で特徴量をソート
sorted_idx = average_feature_importances.argsort()

# 平均の特徴量の重要度をプロット
plt.figure(figsize=(10, 7))
plt.barh(X.columns[sorted_idx], average_feature_importances[sorted_idx])
plt.xlabel("Average Feature Importance across CV")
plt.ylabel("Feature")
plt.title("CrossValidationでのランダムフォレストによる平均特徴量の重要度")

# 画像として保存
plt.savefig("feature_importances_amg_Ctouring.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
print(np.isnan(X).sum(axis=0))

In [ ]:
# #標準化の処理

# from sklearn.preprocessing import StandardScaler
# # モデル構築のためのインポート
# from sklearn.linear_model import Ridge, Lasso, LinearRegression

# # データ分割のためのインポート
# from sklearn.model_selection import train_test_split


# # 特徴量と目的変数を分離
# X = encoded_df.drop(columns='価格')
# y = encoded_df['価格']
# # 標準化
# X = StandardScaler().fit_transform(X)


# # 訓練データとテストデータに分割
# X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# # 各クラスを初期化してdict型変数のmodelsに格納
# models = {
#     'linear': LinearRegression(),
#     'ridge': Ridge(random_state=0),
#     'lasso': Lasso(random_state=0)}

# # 正解率を格納するdict型変数を初期化
# scores = {}

# # 各モデルを順次生成し、正解率を算出して格納
# for model_name, model in models.items():
#     # モデル生成
#     model.fit(X_train, Y_train)
#     # 訓練データの正解率
#     scores[(model_name, 'train')] = model.score(X_train, Y_train)
#     # テストデータの正解率
#     scores[(model_name, 'test')] = model.score(X_test, Y_test)

# # dict型をpandasの1次元リストに変換
# print(pd.Series(scores))


重回帰予測式


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

# 1. データの準備
X = encoded_df.drop(columns='価格')
y = encoded_df['価格']

# 2. データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 重回帰モデルの訓練
reg = LinearRegression()
reg.fit(X_train, y_train)

# 訓練データでの予測
train_predicted = reg.predict(X_train)
# テストデータでの予測
test_predicted = reg.predict(X_test)

# R^2 スコアの計算
train_score = reg.score(X_train, y_train)
test_score = reg.score(X_test, y_test)

print(f"Training R^2 score: {train_score:.2f}")
print(f"Testing R^2 score: {test_score:.2f}")



# 4. 欠損値の確認・補完
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 5. 予測式の表示
print("Coefficients:", reg.coef_)
print("Intercept:", reg.intercept_)
print("予測式: \n")
print(f"価格 = {reg.intercept_:.4f}")
for coef, feature in zip(reg.coef_, X.columns):
    print(f" + ({coef:.4f} * {feature})")

# 6. 予測値の計算
predicted = reg.predict(X)

# 7. 予測値をdummiesに追加
encoded_df['predicted_price'] = predicted
print(encoded_df[['価格', 'predicted_price']])

# 8. 予測値と実際の価格の差分を計算
encoded_df['difference'] = encoded_df['価格'] - encoded_df['predicted_price']
print(encoded_df[['価格', 'predicted_price', 'difference']])

In [ ]:
print(f"価格 = {reg.intercept_:.4f}")
for coef, feature in zip(reg.coef_, X.columns):
    print(f" + ({coef:.4f} * {feature})")

In [ ]:
df1=df.copy()
df1['difference'] = encoded_df['difference']
df1['predicted_price']=encoded_df['predicted_price']

# df1の特定の列を取得
export_data = df1[['名称','年式','グレード名' ,'年' ,'色' ,'Class_Category', 'Url', 'Mission','修復歴'
                   , 'difference','新車価格', '価格', '総額','predicted_price','走行距離']]
# nanを含む行を削除
export_data = export_data.dropna()


# CSVにエクスポート
filename = f"{today_date}_{model_name}_withexpectedPrice.csv"
# df_rev.to_csv(filename, index=False, encoding="utf-8-sig")

export_data.to_csv(f"{filename}", index=False, encoding='utf-8-sig')

#Bokeh　による可視化。　散布図を作成。


"SUb1 更新処理"